In [59]:
import numpy as np
import pandas as pd 
import scipy
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier as RFC, RandomForestRegressor as RFR
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from collections import defaultdict
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

In [43]:
data = pd.read_csv('../Data/Compressed.csv')
data.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [50]:
X = data.drop(['target', 'insuranceclaim', 'charges'], axis = 1)
y_claim = data['insuranceclaim']
y_charge = data['charges']
X_train, X_test, y_train, y_test = train_test_split(X, y_claim, test_size=0.33, random_state=4)

<h2> Predicting whether customer claims insurance </h2>

In [45]:
clf = RFC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9479638009049773

In [46]:
## Removing low correlation features
clf = RFC()
clf.fit(X_train.drop(['sex', 'region'], axis = 1), y_train)
clf.score(X_test.drop(['sex', 'region'], axis = 1), y_test)

0.9705882352941176

<h2> Predicting value of claim </h2>

In [47]:
X = data.drop(['target', 'insuranceclaim', 'charges'], axis = 1)
y = data['charges']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [48]:
clf = RFR()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8221538717292104

In [49]:
## Removing low correlation features
clf = RFR()
clf.fit(X_train.drop(['sex', 'region'], axis = 1), y_train)
clf.score(X_test.drop(['sex', 'region'], axis = 1), y_test)

0.8323627363044895

In [56]:
## Grid search

In [60]:

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [61]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RFC()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_random.fit(train_feature, train_labels)

NameError: name 'train_features' is not defined